In [1]:
import pandas as pd
import numpy as np
import glob
import scipy.stats as stats

In [2]:
featureset=[
'acci_last_infractionage',
'carpoolind_encd',
'classcd_encd',
'driverage',
'drivernumber',
'estimatedannualdistance',
'gooddriverind_encd',
'maritalstatuscd_encd',
'mvrstatus_encd',
'mvrstatusage',
'ratingvalue',
'vehbodytypecd_encd',
'vehicleage',
'vehnumber',
'licenseage',
'gendercd_encd'
]
need_refline_set=['driverage', 
                  'drivernumber', 
                  'estimatedannualdistance', 
                  'ratingvalue',
                  'vehicleage',
                  'vehnumber',
                  'licenseage']

In [3]:
significance_level=0.00001

In [4]:
#models tables files dir
ModelsTblDir='/home/kate/Models/XGB/tbl/'

In [5]:
import glob

In [6]:
#partial dependency
for f in featureset:
    print (f)
    PDFList=glob.glob('%sGL PD *%s.csv'%(ModelsTblDir,f))
    i=0
    Models=[]
    for t in PDFList:
        print(t)
        ModelName=t[32:39]
        Models.append(ModelName)
        df = pd.read_csv(t, index_col=None)
        if i==0:
            if '_encd' in f:
                pdf_df=df[[f,f.replace('_encd',''),'hasclaim','count','pd']]
                pdf_df.columns=[f,f.replace('_encd',''),'hasclaim','count',ModelName]
            elif f in ['estimatedannualdistance','ratingvalue']:
                pdf_df=df[[f,'bin','hasclaim','count','pd']]
                pdf_df.columns=[f,'bin','hasclaim','count',ModelName]                
            else:
                pdf_df=df[[f,'hasclaim','count','pd']]
                pdf_df.columns=[f,'hasclaim','count',ModelName]
        else:
            pdf_df=pd.concat([pdf_df,df[['pd']]], axis=1) 
            pdf_df.rename(columns={'pd':ModelName}, inplace=True)
        i=+1
    pdf_df['Partial Dependency'] = pdf_df[Models].mean(axis=1)
    if f in need_refline_set:
        pdf_df.sort_values(f,inplace=True)
        pdf_df.reset_index(inplace=True)
        pdf_df['pvalue']= np.nan
        pdf_df['statistic']= np.nan
        pdf_df['isRefLine']= np.nan
        for index in range(1,len(pdf_df)):
            row=pdf_df.iloc[index][['Model 2','Model 6','Model 1','Model 5','Model 8','Model 3','Model 0','Model 9','Model 7','Model 4']]
            if index==1:
                RefLine=row
                pdf_df.at[index,'isRefLine']=0
                pdf_df.at[index,'pvalue']=0
                pdf_df.at[index,'statistic']=0
            else:
                t=stats.ttest_ind(RefLine.tolist(),row.tolist())
                pdf_df.at[index,'pvalue']=t.pvalue
                pdf_df.at[index,'statistic']=t.statistic
                if t.pvalue<=significance_level:
                    RefLine=row
                    pdf_df.at[index,'isRefLine']=0.025
                else:
                    pdf_df.at[index,'isRefLine']=0
    pdf_df.to_csv(ModelsTblDir+'%s_GL.csv'%f)

acci_last_infractionage
/home/kate/Models/XGB/tbl/GL PD Model 7 acci_last_infractionage.csv
/home/kate/Models/XGB/tbl/GL PD Model 6 acci_last_infractionage.csv
/home/kate/Models/XGB/tbl/GL PD Model 1 acci_last_infractionage.csv
/home/kate/Models/XGB/tbl/GL PD Model 3 acci_last_infractionage.csv
/home/kate/Models/XGB/tbl/GL PD Model 2 acci_last_infractionage.csv
/home/kate/Models/XGB/tbl/GL PD Model 9 acci_last_infractionage.csv
/home/kate/Models/XGB/tbl/GL PD Model 4 acci_last_infractionage.csv
/home/kate/Models/XGB/tbl/GL PD Model 5 acci_last_infractionage.csv
/home/kate/Models/XGB/tbl/GL PD Model 0 acci_last_infractionage.csv
/home/kate/Models/XGB/tbl/GL PD Model 8 acci_last_infractionage.csv
carpoolind_encd
/home/kate/Models/XGB/tbl/GL PD Model 0 carpoolind_encd.csv
/home/kate/Models/XGB/tbl/GL PD Model 9 carpoolind_encd.csv
/home/kate/Models/XGB/tbl/GL PD Model 6 carpoolind_encd.csv
/home/kate/Models/XGB/tbl/GL PD Model 5 carpoolind_encd.csv
/home/kate/Models/XGB/tbl/GL PD Model 7 

In [7]:
#Feature Importance
FEList=glob.glob('%sGL Feature Importance*.csv'%ModelsTblDir)
Models=[]
fe_df=pd.DataFrame()
i=0;
for t in FEList:
    print(t)
    ModelName=t[48:55]
    df = pd.read_csv(t, index_col=None,header=None)
    df.columns=['Feature',ModelName]
    df['Feature']=pd.to_numeric(df['Feature'].str.extract('(\d+)'))
    df.sort_values(by='Feature',inplace=True)
    if i>0:
        fe_df=pd.merge(fe_df,df, on='Feature')
    else:
        fe_df=df
    i=+1

/home/kate/Models/XGB/tbl/GL Feature Importance Model 2.csv
/home/kate/Models/XGB/tbl/GL Feature Importance Model 9.csv
/home/kate/Models/XGB/tbl/GL Feature Importance Model 6.csv
/home/kate/Models/XGB/tbl/GL Feature Importance Model 7.csv
/home/kate/Models/XGB/tbl/GL Feature Importance Model 8.csv
/home/kate/Models/XGB/tbl/GL Feature Importance Model 5.csv
/home/kate/Models/XGB/tbl/GL Feature Importance Model 1.csv
/home/kate/Models/XGB/tbl/GL Feature Importance Model 0.csv
/home/kate/Models/XGB/tbl/GL Feature Importance Model 3.csv
/home/kate/Models/XGB/tbl/GL Feature Importance Model 4.csv


/home/kate/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # This is added back by InteractiveShellApp.init_path()


In [8]:
def GetFeatureName(i):
    return featureset[int(i)].replace('_encd','')
fe_df['Feature'] = fe_df.apply(lambda row: GetFeatureName(row['Feature']), axis=1)

In [9]:
fe_df['Avg']=fe_df.mean(axis=1)

In [10]:
fe_df.to_csv(ModelsTblDir+'FeatureImportance_GL.csv')